In [ ]:
# import tf and data from mnist
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# one_hot argument means that for each label, the corresponding value is a 1, and the rest are zeroes
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

In [ ]:
# set up three hidden layers, each with 500 nodes
nodes1 = 500
nodes2 = 500
nodes3 = 500
# number of classes, also is the number of nodes in output layer
n_classes = 10
# process data in batches given computing constraints
batch_size = 100
# data variables. note that for x the explicit argument is to filter unexpected data of the wrong size
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [ ]:
def neural_network_model(data):
    # laying out structure of neural network, no computation formulas yet
    # weights are matrices with shape m*n
    # m is number of input nodes, n is number of output nodes for that layer
    # biases are vectors with length l
    # l is number of output nodes for that layer
    # here the weights and biases are initialised randomly
    def layer(i,o):
        return {'weights':tf.Variable(tf.random_normal([i,o])),
               'biases': tf.Variable(tf.random_normal([o]))}
    
    hl1 = layer(784,nodes1)
    hl2 = layer(nodes1, nodes2)
    hl3 = layer(nodes2, nodes3)
    ol = layer(nodes3, n_classes)

    # now defining the mathematical flow of data through the neural network
    # at each layer, output activation value = relu((input x weights) + biases)
    def activate(i,o):
        value = tf.add(tf.matmul(i, o['weights']), o['biases'])
        return tf.nn.relu(value)
    
    l1 = activate(data, hl1)
    l2 = activate(l1, hl2)
    l3 = activate(l2, hl3)
    output = tf.matmul(l3,ol['weights']) + ol['biases']

    return output

In [ ]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    # cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    # minimise cost with Adam Optimizer
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    # each epoch is a training iteration where we calculate the cost and optimize weights and biases
    epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # for each epoch:
        for epoch in range(epochs):
            epoch_loss = 0
            # for each batch:
            for n in range(int(mnist.train.num_examples/batch_size)):
                # calculate cost and optimze
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                n, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)
        # tf.argmax gives index (label) of the highest value (confidence) across the axis 
        # basically turns the matrixes of probabilities into a vector of predicted label values
        # tf.argmax tells us if the predicted label matches the actual label
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        #calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)